### 获取持仓信息

In [1]:
# 给定一个可用的ETF持仓文件地址，返回文件内容
def get_etf_holdings_text(url):
    import requests
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Failed to download file: status code {response.status_code}")
            return 

    except Exception as e:
        print(f"An error occurred: {e}")
        return

### 准备下载地址

In [2]:
# 构建 Direction ETF的所有代码集，以及对应的下载地址
direxion_etfs = {
    "Daily Bull & Bear 3X ETFs": ["MIDU", "TNA", "TZA", "SPXL", "SPXS", "YINN", "YANG", "EURL", "EDC", "EDZ", "MEXX", "KORU", "TYD", "TYO", "TMF", "TMV", "WEBL", "WEBS", "HIBL", "HIBS", "LABU", "LABD", "FAS", "FAZ", "CURE", "NAIL", "DRN", "DRV", "DPST", "RETL", "SOXL", "SOXS"],
    "Daily Bull & Bear 2X ETFs": ["BRZU", "CHAU", "CWEB", "ERX", "ERY", "GUSH", "DRIP", "INDL", "JNUG", "JDST", "NUGT", "DUST", "SPUU", "UBOT", "CLDL", "OOTO", "KLNE", "FNGG", "EVAV"],
    "Daily Bear 1X ETFs": ["SPDN"],
    "Single Stock ETFs": ["AAPU", "AAPD", "TSLL", "TSLS", "AMZU", "AMZD", "GGLL", "GGLS", "MSFU", "MSFD", "NVDU", "NVDD"],
    "Actively Managed Tactical ETFs": ["HCMT"],
    "Non-Leveraged ETFs": ["COM", "HJEN", "MOON", "QQQE", "WFH"]
}
direxion_etfs_all = []
for k,v in direxion_etfs.items():
    direxion_etfs_all += v

direxion_etfs_urls = {x: f"https://www.direxion.com/holdings/{x}.csv" for x in direxion_etfs_all}

In [3]:
# 构建ARK ETF的所有代码集，以及对应的下载地址
ark_etfs_urls = {
    "ARKK": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_INNOVATION_ETF_ARKK_HOLDINGS.csv",
    "ARKW": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_NEXT_GENERATION_INTERNET_ETF_ARKW_HOLDINGS.csv",
    "ARKQ": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_AUTONOMOUS_TECH._&_ROBOTICS_ETF_ARKQ_HOLDINGS.csv",
    "ARKG": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_GENOMIC_REVOLUTION_ETF_ARKG_HOLDINGS.csv",
    "ARKF": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_FINTECH_INNOVATION_ETF_ARKF_HOLDINGS.csv",
    "ARKX": "https://ark-funds.com/wp-content/uploads/funds-etf-csv/ARK_SPACE_EXPLORATION_&_INNOVATION_ETF_ARKX_HOLDINGS.csv",
}

### 根据每家ETF持仓文件的格式，清理数据并写入数据库

In [4]:
from tqdm import tqdm
holdings_data = {}
etf_urls = {**direxion_etfs_urls, **ark_etfs_urls}
for tic, url in tqdm(etf_urls.items()):
    holdings_data[tic] = get_etf_holdings_text(url)

100%|██████████| 76/76 [00:37<00:00,  2.02it/s]


In [5]:
import io
import pandas as pd
holdings_csv = {}
for tic, text in tqdm(holdings_data.items()):
    if tic in ark_etfs_urls:
        try:
            df = pd.read_csv(io.StringIO(text.decode('utf-8'))).iloc[:-1,:]
            df['date'] = df['date'].apply(pd.to_datetime).apply(lambda dt: dt.date().isoformat())
            ark_renamer = {
                'date':'date',
                'fund':'etf_ticker',
                'company':'security_name',
                'ticker':'ticker',
                'cusip':'cusip',
                'shares':'shares',
                'market value ($)':'market_value',
                'weight (%)':'weight'
            }
            df.rename(columns=ark_renamer,inplace=True)
            df['weight'] = df['weight'].apply(lambda x: float(x[:-1])/100)
            df['shares'] = df['shares'].apply(lambda x: int(x.replace(',','')))
            df['market_value'] = df['market_value'].apply(lambda x: float(x.replace(',','').replace('$','')))
            holdings_csv[tic] = df
        except:
            print(f"Failed to parse {tic}")
    elif tic in direxion_etfs_all:
        try:
            df = pd.read_csv(io.StringIO(text.decode('utf-8')),skiprows=5)
            df['date'] = df['TradeDate'].apply(lambda dt: dt.split(' ')[0]).apply(pd.to_datetime).apply(lambda dt: dt.date().isoformat())
            direxion_renamer = {
                'AccountTicker':'etf_ticker',
                'StockTicker':'ticker',
                'SecurityDescription':'security_name',
                'Shares':'shares',
                'Price': 'price',
                'MarketValue':'market_value',
                'Cusip': 'cusip',
                'HoldingsPercent': 'weight'
            }
            df.rename(columns=direxion_renamer,inplace=True)
            df['weight'] = df['weight'].apply(lambda x: x/100)
            holdings_csv[tic] = df.drop(columns=['TradeDate'])
        except:
            print(f"Failed to parse {tic}")

100%|██████████| 76/76 [00:00<00:00, 160.85it/s]


In [6]:
import sqlite3
import datetime
import os
try:
    os.remove('etf_holdings.db')
except:
    pass

# 写入数据库
conn = sqlite3.connect('etf_holdings.db')
pd.concat([df for df in holdings_csv.values()])\
    .assign(update_datetime=datetime.datetime.now().isoformat())\
    .to_sql('etf_holdings',conn,if_exists='append',index=False)

3392

In [7]:
# 测试
conn = sqlite3.connect('etf_holdings.db')
query = """
select etf_ticker, date, count(ticker) as number_of_tickers, count(cusip) as number_of_cusips
from etf_holdings
group by etf_ticker, date
"""
df=pd.read_sql(query,conn)
conn.close()

df

,etf_ticker,date,number_of_tickers,number_of_cusips
0,AAPD,2023-12-29,0,7
1,AAPU,2023-12-29,1,8
2,AMZD,2023-12-29,0,6
3,AMZU,2023-12-29,1,7
4,ARKF,2023-12-29,30,31
...,...,...,...,...
71,WEBL,2023-12-29,41,47
72,WEBS,2023-12-29,0,6
73,WFH,2023-12-29,39,40
74,YANG,2023-12-29,0,5
